In [1]:
import numpy as np
from numpy.linalg import eigh, eig
import pandas as pd
import plotly.express as px
from plotly import graph_objects as go
from scipy.optimize import curve_fit

In [6]:
def lorentz(X, mean, sigma, scale, offset):
    return scale/(np.pi * sigma * (1 + ((X - mean)/sigma)**2)) + offset

def lorentz_shift(X, mean, sigma, scale, offset):
    return -(2 * sigma * (-mean + X)) / (np.pi * (mean**2 + sigma**2 - 2*mean*X + np.power(X, 2))**2)*scale + offset

def lorentz_spread(X, mean, sigma, scale, offset):
    return (mean**2 - 2 * mean * X + np.power(X, 2) - sigma**2)/(np.pi * (mean**2 - 2 * mean * X + np.power(X, 2) + sigma**2)**2) * scale + offset

def both_lorentz(X, mean1, sigma1, scale1, offset1, mean2, sigma2, scale2, offset2):
    return lorentz(X, mean1, sigma1, scale1, offset1) + lorentz_shift(X, mean2, sigma2, scale2, offset2)

class Map:
    def __init__(self, src, x, y, **kwargs):
        raw_data = np.loadtxt(src, delimiter='\t')
        if 'begin' in kwargs and 'end' in kwargs:
            raw_data = raw_data[np.r_[0:1, (kwargs['begin']+2):(kwargs['end']+2)]]
        self.shape = (x, y)
        self.raman_shift = raw_data.T[0][2:]
        self.data_matrix = raw_data[2:].T[1:]
        self.data = pd.DataFrame(raw_data[2:], columns = ['RamanShift'] + list(np.arange(0, x*y, 1)))
        self.map = np.array([raw_data[0][1:], raw_data[1][1:]]).T

        self.cov_matrix = np.cov(self.data_matrix)
        
        eigenvalues, eigenvectors = eigh(self.cov_matrix)
        idx = np.flip(eigenvalues.argsort())
        self.weights = eigenvalues[idx] / sum(eigenvalues)
        self.vectors = eigenvectors.T[idx]
        
        self.base_vectors = np.dot(self.vectors, self.data_matrix)
    
    def reconstruct(self, count):
        return np.dot(self.vectors[:count].T, self.base_vectors[:count])
    
    def plotComponent(self, num):
        fig = px.imshow(np.reshape(self.vectors[num], self.shape), title ='Komponent ' + str(num))
        return fig
    
    def plotPhysicalComponent(self, v):
        img = np.dot(np.array(v), self.vectors[:len(v)])
        fig = px.imshow(np.reshape(img, self.shape), title ='Peak')
        return fig
    
    def plotCovMatrix(self):
        fig = px.imshow(self.cov_matrix, title = 'Macierz kowariancji')
        return fig
    
    def plotWeights(self, cum):
        if cum:
            fig = px.bar(np.log(np.cumsum(self.weights)), title = 'Wagi')
        else:
            fig = px.bar(np.log(self.weights), title = 'Wagi')
        return fig
    
    def plotSpectrums(self, ids, **kwargs):
        fig = go.Figure()
        if 'base' not in kwargs:
            for num in ids:
                fig.add_scatter(x = self.raman_shift, y = self.data_matrix[num], mode='lines', name = 'ID: ' + str(num))
        else:
            r = self.reconstruct(kwargs['base'])
            for num in ids:
                fig.add_scatter(x = self.raman_shift, y = r[num], mode='lines', name = 'ID: ' + str(num))
        fig.update_layout(hovermode="x")
        return fig
    
    def plotSpectrum(self, num, **kwargs):
        return self.plotSpectrums([num], **kwargs)
    
    def plotBaseVectors(self, count):
        fig = go.Figure()
        for num in range(count):
            fig.add_scatter(x = self.raman_shift, y = self.base_vectors[num], mode='lines', name = 'Num: ' + str(num))
        fig.update_layout(hovermode="x")
        return fig
            
    def locToId(self, x, y):
        return np.argmin(list(map(lambda e: np.linalg.norm([x, y] - e), self.map)))
    
    def posToId(self, x, y):
        return self.shape[0] * y + x
    
    def draw_curve(self, curve, params, a, b, **draw):
        fig = go.Figure()
        if 'spectrum' in draw:
            fig.add_scatter(x = self.raman_shift, y = self.data_matrix[draw['spectrum']], mode='lines')
        if 'base_vector' in draw:
            fig.add_scatter(x = self.raman_shift, y = self.base_vectors[draw['base_vector']], mode='lines')
        data_slice = self.data[(self.data['RamanShift'] >= a) & (self.data['RamanShift'] <= b)]['RamanShift']
        fig.add_scatter(mode='lines', x = data_slice, y = curve(data_slice, *params))
        fig.update_layout(hovermode="x")
        return fig
    
    def fit_curve(self, curve, p0, a, b, **plot):
        y = 0
        if 'spectrum' in plot:
            y = self.data_matrix[plot['spectrum'], (self.data['RamanShift'] >= a) & (self.data['RamanShift'] <= b)]
        if 'base_vector' in plot:
            y = self.base_vectors[plot['base_vector'], (self.data['RamanShift'] >= a) & (self.data['RamanShift'] <= b)]
        data_slice = self.data[(self.data['RamanShift'] >= a) & (self.data['RamanShift'] <= b)]['RamanShift']
        
        bounds = 0
        if 'bounds' in plot and plot['bounds'] == 'inf':
            bounds = ((-np.inf, -np.inf, -np.inf, -np.inf), (np.inf, np.inf, np.inf, np.inf))
        else:
            bounds = ((p0[0]-2, -np.inf, -np.inf, -np.inf), (p0[0]+2, np.inf, np.inf, np.inf))
        
        param, cov = curve_fit(curve, data_slice, y, p0=p0, bounds = bounds)
        if 'draw' in plot and plot['draw'] == True:
            self.draw_curve(curve, param, a, b, **plot).show()
        return param[2]

In [7]:
map_a = Map('../data/GA25_10B_532nm_100%_1x3sec_x100_xc1200_A_trojkat1_mapa_10x10um_step_0.3um_data.txt', 34, 34, begin=250, end=450)
# map_b = Map('../data/GA25_10B_532nm_100%_1x3sec_x100_xc1200_B_trojkat3_mapa_10x10um_step_0.3um_data.txt', 34, 34, begin=250, end=450)
# map_c = Map('../data/GA25_10B_532nm_100%_1x3sec_x100_xc1200_C_trojkat2_mapa_20x20um_step_0.3um_data.txt', 68, 68, begin=250, end=450)

In [10]:
map_a.plotWeights(False).show()
# map_b.plotWeights(True).show()
# map_c.plotWeights(True).show()

C:\Users\PJ\AppData\Local\Temp\ipykernel_23488\2973916353.py:53: RuntimeWarning:

invalid value encountered in log



In [ ]:
map_a.plotBaseVectors(5).show()
map_b.plotBaseVectors(5).show()
map_c.plotBaseVectors(5).show()

In [14]:
## Map A
t_matrix = np.zeros((4,4))

draw = False
t_matrix[0][0] = map_a.fit_curve(lorentz, (1392, 2.5, -17 * 10**5, -7000), 1380, 1400, base_vector = 0, draw = draw)

t_matrix[1][0] = map_a.fit_curve(lorentz, (1345, 14,  -5 * 10**5, -10000), 1330, 1360, base_vector = 0, draw = draw)
t_matrix[1][3] = map_a.fit_curve(lorentz, (1345, 16,  3.2 * 10**5, -4000), 1330, 1360, base_vector = 3, draw = draw)

t_matrix[2][1] = map_a.fit_curve(lorentz, (1369, 10,  0.8 * 10**5,  1000), 1355, 1380, base_vector = 1, draw = draw)
t_matrix[2][2] = map_a.fit_curve(lorentz, (1369, 10, -2.4 * 10**5, -1400), 1355, 1380, base_vector = 2, draw = draw)

t_matrix[3][1] = map_a.fit_curve(lorentz_shift, (1392, 2.5, -17 * 10**5, -7000), 1385, 1400, base_vector = 1, draw = draw, bounds = 'inf')
t_matrix[3][2] = map_a.fit_curve(lorentz_shift, (1392, 2.5, -17 * 10**5, -7000), 1385, 1400, base_vector = 2, draw = draw, bounds = 'inf')
t_matrix[3][3] = map_a.fit_curve(lorentz_shift, (1392, 2.5, -17 * 10**5, -7000), 1385, 1400, base_vector = 3, draw = draw, bounds = 'inf')

new_base_vectors = np.dot(map_a.base_vectors[:4].T, np.linalg.inv(t_matrix)).T
new_vectors = np.dot(t_matrix, map_a.vectors[:4])

px.line(x = map_a.raman_shift, y = new_base_vectors[0]).show()
# px.imshow(np.reshape(new_vectors[0], map_a.shape)).show()
px.line(x = map_a.raman_shift, y = new_base_vectors[1]).show()
# px.imshow(np.reshape(new_vectors[1], map_a.shape)).show()
# px.line(x = map_a.raman_shift, y = new_base_vectors[2]).show()
# px.imshow(np.reshape(new_vectors[2], map_a.shape)).show()
# px.line(x = map_a.raman_shift, y = new_base_vectors[3]).show()
px.imshow(np.reshape(new_vectors[3], map_a.shape)).show()

t_matrix

array([[-1650316.79563961,        0.        ,        0.        ,
               0.        ],
       [ -645129.93611166,        0.        ,        0.        ,
          367512.77129684],
       [       0.        ,    61621.73369661,  -240442.3677895 ,
               0.        ],
       [       0.        ,   674604.10153941,    80256.71349783,
          -70178.05019162]])

In [ ]:
map_a.plotSpectrum(map_a.posToId(11, 10))

In [ ]:
## Map B
t_matrix = np.zeros((4,4))

draw = True
t_matrix[0][0] = map_b.fit_curve(lorentz, (1392, 2.5, 17 * 10**5, -7000), 1380, 1400, base_vector = 0, draw = draw)
t_matrix[0][3] = map_b.fit_curve(lorentz, (1392, 2.5, -17 * 10**4, -7000), 1389, 1396, base_vector = 3, draw = draw)


t_matrix[1][0] = map_b.fit_curve(lorentz, (1345, 14,  5 * 10**5, -10000), 1330, 1360, base_vector = 0, draw = draw)
# t_matrix[1][3] = map_b.fit_curve(lorentz, (1345, 14,  -5 * 10**5, 1000), 1335, 1352, base_vector = 3, draw = draw)

t_matrix[2][3] = map_b.fit_curve(lorentz, (1369, 10, 2.4 * 10**5, -1400), 1355, 1380, base_vector = 3, draw = draw)

t_matrix[3][1] = map_b.fit_curve(lorentz_shift, (1392, 2.5, -17 * 10**5, -7000), 1385, 1400, base_vector = 1, draw = draw, bounds = 'inf')
t_matrix[3][2] = map_b.fit_curve(lorentz_shift, (1392, 2.5, -17 * 10**5, -7000), 1385, 1400, base_vector = 2, draw = draw, bounds = 'inf')

# new_base_vectors = np.dot(map_a.base_vectors[:4].T, np.linalg.inv(t_matrix)).T
new_vectors = np.dot(t_matrix, map_b.vectors[:4])

# px.line(x = map_a.raman_shift, y = new_base_vectors[0]).show()
px.imshow(np.reshape(new_vectors[0], map_b.shape)).show()
# px.line(x = map_a.raman_shift, y = new_base_vectors[1]).show()
px.imshow(np.reshape(new_vectors[1], map_b.shape)).show()
# px.line(x = map_a.raman_shift, y = new_base_vectors[2]).show()
px.imshow(np.reshape(new_vectors[2], map_b.shape)).show()
# px.line(x = map_a.raman_shift, y = new_base_vectors[3]).show()
px.imshow(np.reshape(new_vectors[3], map_b.shape)).show()

t_matrix

In [29]:
## Map C
t_matrix = np.zeros((4,4))

draw = True
t_matrix[0][0] = map_c.fit_curve(lorentz, (1392, 2.5, 17 * 10**5, -7000), 1380, 1400, base_vector = 0, draw = draw)

t_matrix[1][0] = map_c.fit_curve(lorentz, (1345, 14,  5 * 10**5, -10000), 1330, 1360, base_vector = 0, draw = draw)
t_matrix[1][1] = map_c.fit_curve(lorentz, (1345, 16,  3.2 * 10**5, -4000), 1330, 1360, base_vector = 1, draw = draw)

# t_matrix[2][3] = map_c.fit_curve(lorentz, (1369, 10,  0.8 * 10**5,  1000), 1355, 1380, base_vector = 3, draw = draw)
map_c.draw_curve(lorentz, (1369, 10, 2.4 * 10**5, 22000), 1355, 1380, base_vector = 3, draw = draw).show()
t_matrix[2][3] = 22000

t_matrix[3][1] = map_c.fit_curve(lorentz_shift, (1392, 2.5, 17 * 10**5, -7000), 1385, 1400, base_vector = 1, draw = draw, bounds = 'inf')
t_matrix[3][2] = map_c.fit_curve(lorentz_shift, (1392, 2.5, -17 * 10**5, -7000), 1385, 1400, base_vector = 2, draw = draw, bounds = 'inf')
t_matrix[3][3] = map_c.fit_curve(lorentz_shift, (1392, 2.5, -17 * 10**5, -7000), 1385, 1400, base_vector = 3, draw = draw, bounds = 'inf')

# new_base_vectors = np.dot(map_a.base_vectors[:4].T, np.linalg.inv(t_matrix)).T
new_vectors = np.dot(t_matrix, map_c.vectors[:4])

# px.line(x = map_a.raman_shift, y = new_base_vectors[0]).show()
px.imshow(np.reshape(new_vectors[0], map_c.shape)).show()
# px.line(x = map_a.raman_shift, y = new_base_vectors[1]).show()
px.imshow(np.reshape(new_vectors[1], map_c.shape)).show()
# px.line(x = map_a.raman_shift, y = new_base_vectors[2]).show()
px.imshow(np.reshape(new_vectors[2], map_c.shape)).show()
# px.line(x = map_a.raman_shift, y = new_base_vectors[3]).show()
px.imshow(np.reshape(new_vectors[3], map_c.shape)).show()

t_matrix

array([[ 5077099.54876151,        0.        ,        0.        ,
               0.        ],
       [ 1506878.44911876,   727989.90315262,        0.        ,
               0.        ],
       [       0.        ,        0.        ,        0.        ,
           22000.        ],
       [       0.        ,  1063326.41294676, -1612406.4280187 ,
         -328784.05414607]])

In [30]:
map_c.plotSpectrum(map_c.posToId(14, 42))